**This notebook is an exercise in the [Advanced SQL](https://www.kaggle.com/learn/advanced-sql) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/analytic-functions).**

---


# Introduction

Here, you'll use window functions to answer questions about the [Chicago Taxi Trips](https://www.kaggle.com/chicago/chicago-taxi-trips-bq) dataset.

Before you get started, run the code cell below to set everything up.

In [1]:
# Get most recent checking code
!pip install -U -t /kaggle/working/ git+https://github.com/Kaggle/learntools.git
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql_advanced.ex2 import *
print("Setup Complete")

  Cloning https://github.com/Kaggle/learntools.git to /tmp/pip-req-build-wt2crzkl
  Running command git clone --filter=blob:none --quiet https://github.com/Kaggle/learntools.git /tmp/pip-req-build-wt2crzkl
  Resolved https://github.com/Kaggle/learntools.git to commit a0c08ac1fa73dd42ba4aecf2cce045752f14f72b
  Preparing metadata (setup.py) ... done
  Created wheel for learntools: filename=learntools-0.3.5-py3-none-any.whl size=269685 sha256=dede9a53ed5bd3b60dfcd40796e9a1df5e0b90b7f8f86a6f5f15dccccb9c4732
  Stored in directory: /tmp/pip-ephem-wheel-cache-aaw7d8vv/wheels/50/16/ba/3e0ec276f3238de9e1c59751f34c7c3ea4a3f561af10c4fd0c
Successfully built learntools
Using Kaggle's public dataset BigQuery integration.


/usr/local/lib/python3.11/dist-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Setup Complete


The following code cell fetches the `taxi_trips` table from the `chicago_taxi_trips` dataset. We also preview the first five rows of the table.  You'll use the table to answer the questions below.

In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_taxi_trips" dataset
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "taxi_trips" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,9f4d762848799be329e007ae695297ddf2ec9e68,53b9f89457dbe76abd2c5d0d7f00e0c2bf48a1e1035122...,2019-09-09 15:00:00+00:00,2019-09-09 15:15:00+00:00,1279,14.39,<NA>,<NA>,<NA>,<NA>,...,24.0,60.75,Credit Card,Flash Cab,NaN,NaN,None,NaN,NaN,None
1,9f28593d41d91fcf46c81ab9395c2dc8a6a26c9a,a7108907c04f278905344870b79e80244259d14f1335b5...,2019-09-04 07:15:00+00:00,2019-09-04 07:45:00+00:00,1920,14.40,<NA>,<NA>,<NA>,<NA>,...,0.0,44.20,Credit Card,Star North Management LLC,NaN,NaN,None,NaN,NaN,None
2,9dd67852ff43243a3121d24ea21ac5f84dd798a1,81d652b4d1a83430b2ecbdd8f126b135f9f37b6115b7a5...,2019-09-05 07:15:00+00:00,2019-09-05 07:30:00+00:00,459,1.00,<NA>,<NA>,<NA>,<NA>,...,0.0,6.50,Cash,Taxi Affiliation Service Yellow,NaN,NaN,None,NaN,NaN,None
3,9e870b0b436c1124f41a4c1cef0457c5196fb6e3,22c06e3ec9587da0d7536065869f2da9965faf286be583...,2019-09-18 11:30:00+00:00,2019-09-18 11:30:00+00:00,450,1.10,<NA>,<NA>,<NA>,<NA>,...,0.0,6.50,Cash,Flash Cab,NaN,NaN,None,NaN,NaN,None
4,9db4f38602989a10d3cf516e42098492a4c89e3f,d651380c76d136dff3d8f30a44f5f5b017dbe925676ba6...,2019-09-03 15:45:00+00:00,2019-09-03 15:45:00+00:00,254,0.77,<NA>,<NA>,<NA>,<NA>,...,0.0,5.25,Cash,Flash Cab,NaN,NaN,None,NaN,NaN,None


# Exercises

### 1) How can you predict the demand for taxis?

Say you work for a taxi company, and you're interested in predicting the demand for taxis.  Towards this goal, you'd like to create a plot that shows a rolling average of the daily number of taxi trips.  Amend the (partial) query below to return a DataFrame with two columns:
- `trip_date` - contains one entry for each date from January 1, 2016, to March 31, 2016.
- `avg_num_trips` - shows the average number of daily trips, calculated over a window including the value for the current date, along with the values for the preceding 3 days and the following 3 days, as long as the days fit within the three-month time frame.  For instance, when calculating the value in this column for January 3, 2016, the window will include the number of trips for the preceding 2 days, the current date, and the following 3 days.

This query is partially completed for you, and you need only write the part that calculates the `avg_num_trips` column.  Note that this query uses a common table expression (CTE); if you need to review how to use CTEs, you're encouraged to check out [this tutorial](https://www.kaggle.com/dansbecker/as-with) in the [Intro to SQL](https://www.kaggle.com/learn/intro-to-sql) course.

In [3]:
# Fill in the blank below
avg_num_trips_query = """
                      WITH trips_by_day AS
                      (
                      SELECT DATE(trip_start_timestamp) AS trip_date,
                          COUNT(*) as num_trips
                      FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                      WHERE trip_start_timestamp > '2016-01-01' AND trip_start_timestamp < '2016-04-01'
                      GROUP BY trip_date
                      ORDER BY trip_date
                      )
                      SELECT trip_date,
                          AVG(num_trips)
                          OVER (
                               ORDER BY trip_date
                               ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING 
                               ) AS avg_num_trips
                      FROM trips_by_day
                      """
# Check your answer
q_1.check()

/usr/local/lib/python3.11/dist-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,trip_date,avg_num_trips
0,2016-03-16,98023.857143
1,2016-03-07,92348.571429
2,2016-03-27,84715.000000
3,2016-01-13,85459.285714
4,2016-01-16,84642.285714


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [4]:
# Lines below will give you a hint or solution code
#q_1.hint()
q_1.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

avg_num_trips_query = """
                      WITH trips_by_day AS
                      (
                      SELECT DATE(trip_start_timestamp) AS trip_date,
                          COUNT(*) as num_trips
                      FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                      WHERE trip_start_timestamp > '2016-01-01' AND trip_start_timestamp < '2016-04-01'
                      GROUP BY trip_date
                      )
                      SELECT trip_date,
                          AVG(num_trips) 
                          OVER (
                               ORDER BY trip_date
                               ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING
                               ) AS avg_num_trips
                      FROM trips_by_day
                      """

```

### 2) Can you separate and order trips by community area?

The query below returns a DataFrame with three columns from the table: `pickup_community_area`, `trip_start_timestamp`, and `trip_end_timestamp`.  

Amend the query to return an additional column called `trip_number` which shows the order in which the trips were taken from their respective community areas.  So, the first trip of the day originating from community area 1 should receive a value of 1; the second trip of the day from the same area should receive a value of 2.  Likewise, the first trip of the day from community area 2 should receive a value of 1, and so on.

Note that there are many numbering functions that can be used to solve this problem (depending on how you want to deal with trips that started at the same time from the same community area); to answer this question, please use the **RANK()** function.

In [5]:
# Amend the query below
trip_number_query = """
                        SELECT
                        pickup_community_area,
                        trip_start_timestamp,
                        trip_end_timestamp,
                        RANK() 
                        OVER (
                            PARTITION BY pickup_community_area 
                            ORDER BY trip_start_timestamp
                        ) AS trip_number
                    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                    WHERE DATE(trip_start_timestamp) = '2013-10-03' 
                    """

# Check your answer
q_2.check()

/usr/local/lib/python3.11/dist-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,pickup_community_area,trip_start_timestamp,trip_end_timestamp,trip_number
0,1,2013-10-03 00:00:00+00:00,2013-10-03 00:15:00+00:00,1
1,1,2013-10-03 00:15:00+00:00,2013-10-03 00:15:00+00:00,2
2,1,2013-10-03 00:30:00+00:00,2013-10-03 01:00:00+00:00,3
3,1,2013-10-03 00:30:00+00:00,2013-10-03 01:00:00+00:00,3
4,1,2013-10-03 00:30:00+00:00,2013-10-03 00:45:00+00:00,3


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [6]:
# Lines below will give you a hint or solution code
#q_2.hint()
q_2.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

trip_number_query = """
                    SELECT pickup_community_area,
                        trip_start_timestamp,
                        trip_end_timestamp,
                        RANK()
                            OVER (
                                  PARTITION BY pickup_community_area
                                  ORDER BY trip_start_timestamp
                                 ) AS trip_number
                    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                    WHERE DATE(trip_start_timestamp) = '2013-10-03' 
                    """

trip_number_result = client.query(trip_number_query).result().to_dataframe()

```

### 3) How much time elapses between trips?

The (partial) query in the code cell below shows, for each trip in the selected time frame, the corresponding `taxi_id`, `trip_start_timestamp`, and `trip_end_timestamp`. 

Your task in this exercise is to edit the query to include an additional `prev_break` column that shows the length of the break (in minutes) that the driver had before each trip started (this corresponds to the time between `trip_start_timestamp` of the current trip and `trip_end_timestamp` of the previous trip).  Partition the calculation by `taxi_id`, and order the results within each partition by `trip_start_timestamp`.

Some sample results are shown below, where all rows correspond to the same driver (or `taxi_id`).  Take the time now to make sure that the values in the `prev_break` column make sense to you!

![first_commands](https://storage.googleapis.com/kaggle-media/learn/images/qjvQzg8.png)

Note that the first trip of the day for each driver should have a value of **NaN** (not a number) in the `prev_break` column.

In [7]:
# Fill in the blanks below
break_time_query = """
                   SELECT taxi_id,
                       trip_start_timestamp,
                       trip_end_timestamp,
                       TIMESTAMP_DIFF(
                           trip_start_timestamp, 
                           LAG(trip_end_timestamp) OVER (
            PARTITION BY taxi_id 
            ORDER BY trip_start_timestamp
        ),
                           MINUTE) as prev_break
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE DATE(trip_start_timestamp) = '2013-10-03' 
                   """

# Check your answer
q_3.check()

/usr/local/lib/python3.11/dist-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,taxi_id,trip_start_timestamp,trip_end_timestamp,prev_break
0,145960422e0a08c023dc4616dcb30daeff190b424755b0...,2013-10-03 08:30:00+00:00,2013-10-03 08:30:00+00:00,285
1,15c03618b76c932fd23d3054627cdbbd21323bd13316f4...,2013-10-03 08:30:00+00:00,2013-10-03 08:45:00+00:00,330
2,15c03618b76c932fd23d3054627cdbbd21323bd13316f4...,2013-10-03 18:30:00+00:00,2013-10-03 19:15:00+00:00,240
3,342509ce9f04d14185cf073c4cd71ea079413a622ea3c2...,2013-10-03 07:00:00+00:00,2013-10-03 07:15:00+00:00,375
4,342509ce9f04d14185cf073c4cd71ea079413a622ea3c2...,2013-10-03 18:15:00+00:00,2013-10-03 18:15:00+00:00,405


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [8]:
# Lines below will give you a hint or solution code
#q_3.hint()
#q_3.solution()

# Keep going

Move on to learn how to query **[nested and repeated data](https://www.kaggle.com/alexisbcook/nested-and-repeated-data)**.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/advanced-sql/discussion) to chat with other learners.*